# 评估查询结果

## 数据准备

In [1]:
%%time
%%capture

%env DATA_PATH=simple-data
!rm -rf $DATA_PATH && mkdir -p $DATA_PATH

import os
data_path = os.getenv('DATA_PATH')

!pip install trafilatura

CPU times: user 17 ms, sys: 7.47 ms, total: 24.5 ms
Wall time: 3.74 s


In [2]:
%%time

from llama_index.readers.web import TrafilaturaWebReader

items=[
    "颐和园",
    "故宫",
    "北海公园",
]

documents = TrafilaturaWebReader().load_data(
    [ f"https://baike.baidu.com/item/{item}" for item in items]
)

len(documents)

CPU times: user 3.38 s, sys: 367 ms, total: 3.75 s
Wall time: 3.65 s


3

In [3]:
%%time

import os

documents_data=[[items[index], documents[index].text] for index,item in enumerate(items)]

for data in documents_data:
    with open(os.path.join(f'{data_path}', f"{data[0]}.txt"), "w", encoding="utf-8") as file:
        file.write(data[1])
    

CPU times: user 1.61 ms, sys: 0 ns, total: 1.61 ms
Wall time: 1.47 ms


## 模型准备

In [4]:
%%time

from llama_index.llms.openai_like import OpenAILike
from llama_index.embeddings.ollama import OllamaEmbedding
from llama_index.core import Settings

Settings.llm =OpenAILike(
    model="qwen2", 
    api_base="http://monkey:11434/v1", 
    api_key="ollama",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

Settings.embed_model =OllamaEmbedding(
    model_name="quentinz/bge-large-zh-v1.5",
    base_url="http://monkey:11434",
    ollama_additional_kwargs={"mirostat": 0}, # -mirostat N 使用 Mirostat 采样。
)

Settings.chunk_size=128
Settings.chunk_overlap=10


evaluating_llm=OpenAILike(
    model='gpt-4-turbo', 
    api_base="http://ape:3000/v1", 
    api_key="sk-bJP6QSnUfjAYeYeE505d3eBf63A643BeB0B8E350Df9b7750",
    is_chat_model=True,
    temperature=0.1,
    request_timeout=60.0
)

CPU times: user 931 ms, sys: 31.9 ms, total: 962 ms
Wall time: 962 ms


## 嵌套异步的支持

In [5]:
%%time

import nest_asyncio
nest_asyncio.apply()

CPU times: user 868 µs, sys: 83 µs, total: 951 µs
Wall time: 740 µs


## 生成索引

In [6]:
%%time

from llama_index.core import VectorStoreIndex

index = VectorStoreIndex.from_documents(documents=documents)

CPU times: user 2.27 s, sys: 154 ms, total: 2.43 s
Wall time: 48.1 s


## 使用索引查询

In [7]:
%%time

query_engine = index.as_query_engine()
response = query_engine.query(
    "北海公园在哪儿?"
)
response.response

CPU times: user 104 ms, sys: 8.34 ms, total: 113 ms
Wall time: 1.63 s


'北海公园位于北京市西城区文津街1号。'

## 对查询结果的评估

In [8]:
%%time

from llama_index.core.evaluation import FaithfulnessEvaluator

evaluator = FaithfulnessEvaluator(llm=evaluating_llm)

query_engine = index.as_query_engine()
response = query_engine.query(
    "北海公园在哪儿?"
)

eval_result = evaluator.evaluate_response(response=response)
print(str(eval_result.passing))

True
CPU times: user 88.1 ms, sys: 0 ns, total: 88.1 ms
Wall time: 2.7 s


In [9]:
eval_result

EvaluationResult(query=None, contexts=['收藏\n查看我的收藏\n0有用+1\n北海公园（Beihai Park），位于北京市西城区文津街1号，东邻景山公园，南濒中南海，北连什刹海，全园占地68.2万平方米（其中水域面积38.9万平方米，陆地面积29.3万平方米），1925年开放为公园。', '北海公园\n- 外文名\n- Beihai Park\n- 地理位置\n- 北京市西城区文津街1号\n- 气候条件\n- 温带季风气候\n- 开放时间\n- 4月1日-10月31日 06:00-21:00；11月1日-次年3月31日 06:30-20:00 [1]\n- 景点级别\n- AAAA级\n-'], response='北海公园位于北京市西城区文津街1号。', passing=True, feedback='YES', score=1.0, pairwise_source=None, invalid_result=False, invalid_reason=None)

## 对检索结果的评估

In [18]:
%%time

from llama_index.core.evaluation import RetrieverEvaluator

retriever = index.as_retriever(similarity_top_k=2)

retriever_evaluator = RetrieverEvaluator.from_metric_names(
    ["mrr", "hit_rate"], retriever=retriever
)

eval_result=retriever_evaluator.evaluate(
    query="北海公园在哪儿?", 
    expected_ids=[
        "cdd17ca3-a9d6-4ef6-8ad2-5697858fd9cd", 
    ],
)

CPU times: user 19.6 ms, sys: 277 µs, total: 19.8 ms
Wall time: 125 ms


In [19]:
eval_result

RetrievalEvalResult(query='北海公园在哪儿?', expected_ids=['cdd17ca3-a9d6-4ef6-8ad2-5697858fd9cd'], expected_texts=None, retrieved_ids=['cdd17ca3-a9d6-4ef6-8ad2-5697858fd9cd', 'c2a4da8b-4bfb-4902-a8d8-26aec1fe445e'], retrieved_texts=['收藏\n查看我的收藏\n0有用+1\n北海公园（Beihai Park），位于北京市西城区文津街1号，东邻景山公园，南濒中南海，北连什刹海，全园占地68.2万平方米（其中水域面积38.9万平方米，陆地面积29.3万平方米），1925年开放为公园。', '北海公园\n- 外文名\n- Beihai Park\n- 地理位置\n- 北京市西城区文津街1号\n- 气候条件\n- 温带季风气候\n- 开放时间\n- 4月1日-10月31日 06:00-21:00；11月1日-次年3月31日 06:30-20:00 [1]\n- 景点级别\n- AAAA级\n-'], mode=<RetrievalEvalMode.TEXT: 'text'>, metric_dict={'mrr': RetrievalMetricResult(score=1.0, metadata={}), 'hit_rate': RetrievalMetricResult(score=1.0, metadata={})})

In [20]:
eval_result.retrieved_texts[0]

'收藏\n查看我的收藏\n0有用+1\n北海公园（Beihai Park），位于北京市西城区文津街1号，东邻景山公园，南濒中南海，北连什刹海，全园占地68.2万平方米（其中水域面积38.9万平方米，陆地面积29.3万平方米），1925年开放为公园。'

In [21]:
eval_result.metric_dict['hit_rate'].score

1.0